In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [8]:
print(vectordb._collection.count())

208


In [9]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [10]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [11]:
from langchain.chains import RetrievalQA

In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [13]:
result = qa_chain({"query": question})

/tmp/ipykernel_7861/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [14]:
result["result"]

'The major topics for this class include machine learning, statistics, and algebra. Additionally, there will be discussions on extensions of the material covered in the main lectures.'

In [15]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [16]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [17]:
question = "Is probability a class topic?"

In [18]:
result = qa_chain({"query": question})

In [19]:
result["result"]

'Yes, probability is a topic assumed to be familiar to students in the class, as mentioned by the instructor. Thanks for asking!'

In [20]:
result["source_documents"][0]

Document(metadata={'author': '', 'creator': 'PScript5.dll Version 5.2.2', 'page_label': '5', 'source': 'MachineLearning-Lecture01.pdf', 'title': '', 'creationdate': '2008-07-11T11:25:23-07:00', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'moddate': '2008-07-11T11:25:23-07:00', 'total_pages': 22, 'page': 4}, page_content="of this class will not be very programming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octave. I'll say a bit more about that later.  \nI also assume familiarity with basic probability and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what random variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites, sort o

In [21]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [22]:
result = qa_chain_mr({"query": question})

In [23]:
result["result"]

'Yes, probability is a class topic in the document. The instructor mentions using a probabilistic interpretation to derive the next learning algorithm, which will be the first classification algorithm.'

In [30]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "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" # replace dots with your api key

In [31]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

'The prerequisites of basic probability and statistics, as well as basic linear algebra, are needed for the class because the course will involve programming in MATLAB or Octave, and understanding concepts like random variables, expectation, variance, matrices, vectors, matrix multiplication, matrix inverse, and eigenvectors are essential for the material covered in the course. Additionally, the instructor assumes that all students have a basic knowledge of computer science and basic computer skills and principles to ensure they can understand and apply the concepts taught in the machine learning class.'

In [26]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'The original answer still adequately addresses the question about probability being a class topic in the course. The additional context provided about using discussion sections to review statistics and algebra as refreshers, as well as covering extensions to the main lecture material, does not directly impact the relevance of probability as a class topic. Therefore, the original answer stands as is.'

In [27]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [28]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic in the course being described. The instructor assumes familiarity with basic probability and statistics, so it is likely that probability concepts will be covered in the class.'

In [29]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites mentioned in the context are needed because the course assumes familiarity with basic concepts in probability and statistics, as well as basic linear algebra. Understanding these concepts is essential for grasping the material covered in the machine learning course.'